# Create Hypothesis Nanopublications (Super-Pattern)

This notebook generates **hypothesis nanopublications** using the super-pattern template.

The super-pattern expresses formal hypotheses as: *In the context of [context], [subject] [qualifier] [relation] [object]*

**Template:** [Expressing a general hypothesis with a super-pattern](http://purl.org/np/RABf4z3vjQrAOkGQRHvW82MrZTqtAsGWU7C9W1niaMOPw)

**Reference:** [Kuhn et al. (2021) - Semantic micro-contributions with decentralized nanopublication services](https://doi.org/10.1145/3460210.3493561)

---
## Configuration

In [ ]:
# Path to your hypothesis config file
CONFIG_FILE = "../config/dggs_hypothesis.json"

---
## Setup

In [ ]:
import json
import sys
from pathlib import Path
from datetime import datetime, timezone

# Add parent directory to path for imports
sys.path.insert(0, str(Path('.').resolve().parent))

try:
    from nanopub_utils import load_config, get_timestamp, escape_literal
    print("✓ Loaded nanopub_utils")
except ImportError:
    print("⚠ nanopub_utils not found, using inline functions")
    
    def load_config(path):
        with open(path, 'r', encoding='utf-8') as f:
            return json.load(f)
    
    def get_timestamp():
        return datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + "Z"
    
    def escape_literal(text):
        return text.replace('\\', '\\\\').replace('"', '\\"').replace('\n', '\\n').replace('\r', '\\r')

# Output directory
OUTPUT_DIR = Path("../output/hypothesis")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
print(f"✓ Output directory: {OUTPUT_DIR}")

---
## Load Configuration

In [ ]:
config = load_config(CONFIG_FILE)

metadata = config['metadata']
nanopubs = config['nanopublications']

print(f"✓ Loaded configuration")
print(f"  Source paper: {metadata['source_paper']['title'][:60]}...")
print(f"  Creator: {metadata['creator_name']}")
print(f"  Hypotheses to create: {len(nanopubs)}")
print()
for np_config in nanopubs:
    print(f"  - {np_config['id']}: {np_config['label'][:60]}...")

---
## Template URIs

In [ ]:
# Hypothesis super-pattern template
HYPOTHESIS_TEMPLATE = "http://purl.org/np/RABf4z3vjQrAOkGQRHvW82MrZTqtAsGWU7C9W1niaMOPw"

# Standard templates
PROVENANCE_TEMPLATE = "http://purl.org/np/RANwQa4ICWS5SOjw7gp99nBpXBasapwtZF1fIM3H2gYTM"
PUBINFO_TEMPLATE_1 = "http://purl.org/np/RAA2MfqdBCzmz9yVWjKLXNbyfBNcwsMmOqcNUxkk1maIM"
PUBINFO_TEMPLATE_2 = "http://purl.org/np/RAh1gm83JiG5M6kDxXhaYT1l49nCzyrckMvTzcPn-iv90"
PUBINFO_TEMPLATE_3 = "http://purl.org/np/RAjpBMlw3owYhJUBo3DtsuDlXsNAJ8cnGeWAutDVjuAuI"

# Super-pattern namespace
SP = "https://w3id.org/linkflows/superpattern/terms/"

print("✓ Template URIs configured")

---
## Generate Nanopublications

In [ ]:
def generate_hypothesis_trig(np_config, metadata):
    """Generate a hypothesis nanopublication using the super-pattern template.
    
    Super-pattern structure:
    - Context Class: The domain where the hypothesis applies
    - Subject Class: What type of thing is the subject
    - Relation: The relationship type
    - Object Class: What type of thing is the object  
    - Qualifier (optional): Frequency qualifier
    """
    
    creator_orcid = metadata['creator_orcid']
    creator_name = metadata['creator_name']
    
    label = escape_literal(np_config['label'])
    context_class = np_config['context_class']
    subject_class = np_config['subject_class']
    relation = np_config['relation']
    object_class = np_config['object_class']
    qualifier = np_config.get('qualifier')  # Optional
    
    timestamp = get_timestamp()
    
    trig = f'''@prefix this: <http://purl.org/nanopub/temp/np/> .
@prefix sub: <http://purl.org/nanopub/temp/np#> .
@prefix np: <http://www.nanopub.org/nschema#> .
@prefix dct: <http://purl.org/dc/terms/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix orcid: <https://orcid.org/> .
@prefix nt: <https://w3id.org/np/o/ntemplate/> .
@prefix npx: <http://purl.org/nanopub/x/> .
@prefix sp: <https://w3id.org/linkflows/superpattern/terms/> .

sub:Head {{
  this: a np:Nanopublication ;
    np:hasAssertion sub:assertion ;
    np:hasProvenance sub:provenance ;
    np:hasPublicationInfo sub:pubinfo .
}}

sub:assertion {{
  sub:spi a sp:SuperPatternInstance ;
    rdfs:label "{label}" ;
    sp:hasContextClass <{context_class['uri']}> ;
    sp:hasSubjectClass <{subject_class['uri']}> ;
    sp:hasRelation <{relation['uri']}> ;
    sp:hasObjectClass <{object_class['uri']}>
'''
    
    # Add qualifier if provided
    if qualifier:
        trig += f'''    ; sp:hasQualifier <{qualifier['uri']}>
'''
    
    trig += '''    .
}

'''
    
    # Provenance
    trig += f'''sub:provenance {{
  sub:assertion prov:wasAttributedTo orcid:{creator_orcid} .
}}

sub:pubinfo {{
  this: dct:created "{timestamp}"^^xsd:dateTime ;
    dct:creator orcid:{creator_orcid} ;
    npx:introduces sub:spi ;
    nt:wasCreatedFromProvenanceTemplate <{PROVENANCE_TEMPLATE}> ;
    nt:wasCreatedFromPubinfoTemplate <{PUBINFO_TEMPLATE_1}> , <{PUBINFO_TEMPLATE_2}> , <{PUBINFO_TEMPLATE_3}> ;
    nt:wasCreatedFromTemplate <{HYPOTHESIS_TEMPLATE}> .
'''
    
    # Add labels for classes (for display in Nanodash)
    trig += f'''
  <{context_class['uri']}> rdfs:label "{escape_literal(context_class['label'])}" .
  <{subject_class['uri']}> rdfs:label "{escape_literal(subject_class['label'])}" .
  <{object_class['uri']}> rdfs:label "{escape_literal(object_class['label'])}" .
'''
    
    trig += '''}}
'''
    
    return trig

print("✓ Generation function defined")

In [ ]:
# Generate all nanopublications
generated_files = []

for np_config in nanopubs:
    np_id = np_config['id']
    trig_content = generate_hypothesis_trig(np_config, metadata)
    
    output_file = OUTPUT_DIR / f"{np_id}.trig"
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(trig_content)
    
    generated_files.append(output_file)
    print(f"✓ Generated: {output_file.name}")
    print(f"    {np_config['label'][:70]}...")

print(f"\n✓ Generated {len(generated_files)} hypothesis nanopublication(s)")

---
## Preview Generated Content

In [ ]:
# Preview the first generated file
if generated_files:
    print(f"Preview of {generated_files[0].name}:")
    print("=" * 70)
    with open(generated_files[0], 'r') as f:
        print(f.read())

---
## Summary

In [ ]:
print("=" * 70)
print("SUMMARY")
print("=" * 70)
print(f"Config file:  {CONFIG_FILE}")
print(f"Output dir:   {OUTPUT_DIR}")
print(f"Generated:    {len(generated_files)} file(s)")
print()
print("Hypotheses:")
for np_config in nanopubs:
    qual = np_config.get('qualifier', {}).get('label', 'no qualifier')
    print(f"  [{np_config['id']}]")
    print(f"    Context: {np_config['context_class']['label']}")
    print(f"    Subject: {np_config['subject_class']['label']}")
    print(f"    Relation: {np_config['relation']['label']} ({qual})")
    print(f"    Object: {np_config['object_class']['label']}")
    print()
print("Next steps:")
print("  1. Review the generated .trig files")
print("  2. Sign: nanopub sign <filename>.trig")
print("  3. Publish: nanopub publish <filename>.signed.trig")

---
## JSON Configuration Template

```json
{
  "metadata": {
    "source_paper": {
      "title": "Your Paper Title",
      "doi": "10.xxxx/xxxxx"
    },
    "creator_orcid": "0000-0002-XXXX-XXXX",
    "creator_name": "Your Name",
    "is_part_of": {
      "uri": null,
      "label": null
    }
  },
  "nanopublications": [
    {
      "id": "hypothesis_unique_id",
      "label": "Human-readable hypothesis statement",
      "context_class": {
        "uri": "http://www.wikidata.org/entity/QXXX",
        "label": "Context domain"
      },
      "subject_class": {
        "uri": "http://www.wikidata.org/entity/QXXX",
        "label": "Subject type"
      },
      "relation": {
        "uri": "https://w3id.org/linkflows/superpattern/terms/causes",
        "label": "causes"
      },
      "object_class": {
        "uri": "http://www.wikidata.org/entity/QXXX",
        "label": "Object type"
      },
      "qualifier": {
        "uri": "https://w3id.org/linkflows/superpattern/terms/generallyQualifier",
        "label": "generally (≥90% of cases)"
      },
      "note": "Optional note about the hypothesis"
    }
  ]
}
```

### Available Relations

| Relation | URI | Meaning |
|----------|-----|--------|
| affects | `sp:affects` | Positively or negatively affects |
| causes | `sp:causes` | Causes existence of |
| contributesTo | `sp:contributesTo` | Positively affects |
| inhibits | `sp:inhibits` | Negatively affects |
| enables | `sp:enables` | Causes activity to happen |
| prevents | `sp:prevents` | Causes activity not to happen |
| increases | `sp:increases` | Causes value to increase |
| decreases | `sp:decreases` | Causes value to decrease |
| requires | `sp:requires` | Would not exist without |
| hasLargerValueThan | `sp:hasLargerValueThan` | Quantitative comparison |
| hasSmallerValueThan | `sp:hasSmallerValueThan` | Quantitative comparison |
| hasSameValueAs | `sp:hasSameValueAs` | Equality |
| cooccursWith | `sp:cooccursWith` | Spatiotemporal proximity |
| follows | `sp:follows` | Temporal sequence |
| includes | `sp:includes` | Spatiotemporal inclusion |

### Available Qualifiers

| Qualifier | URI | Meaning |
|-----------|-----|--------|
| always | `sp:alwaysQualifier` | 100% of cases |
| generally | `sp:generallyQualifier` | ≥90% of cases |
| mostly | `sp:mostlyQualifier` | ≥50% of cases |
| frequently | `sp:frequentlyQualifier` | ≥10% of cases |
| sometimes | `sp:sometimesQualifier` | ≥0.1% of cases |
| never | `sp:neverQualifier` | 0% of cases |

### Special Context

Use `https://w3id.org/linkflows/superpattern/terms/UniversalContext` when no specific context applies.